## Importando pacotes e arquivos

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
#from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [49]:
labels_1942_36 = np.load('imagens/1942-36/labels.npy')
labels_1942_38 = np.load('imagens/1942-38/labels.npy')
labels_1944_24 = np.load('imagens/1944/labels.npy')
labels_1946_44 = np.load('imagens/1946/labels.npy')
labels_1935_47 = np.load('imagens/1935/labels.npy')
labels_1945_38 = np.load('imagens/1945/labels.npy')
labels_1929_101_1 = np.load('imagens/1929/labels-1.npy')
labels_1960_13 = np.load('imagens/1960/labels.npy')
labels_gerados = np.load('imagens/teste - gerado/labels.npy')
labels_1959_53 = np.load('imagens/1950/labels-1.npy')

digits_1942_36 = np.load('imagens/1942-36/digits.npy')
digits_1942_38 = np.load('imagens/1942-38/digits.npy')
digits_1944_24 = np.load('imagens/1944/digits.npy')
digits_1946_44 = np.load('imagens/1946/digits.npy')
digits_1935_47 = np.load('imagens/1935/digits.npy')
digits_1945_38 = np.load('imagens/1945/digits.npy')
digits_1929_101_1 = np.load('imagens/1929/digits-1.npy')
digits_1960_13 = np.load('imagens/1960/digits.npy')
digits_gerados = np.load('imagens/teste - gerado/digits.npy')
digits_1959_53 = np.load('imagens/1950/digits-1.npy')

## Definindo e compilando NN

In [54]:
def identity_block(x,filter):
    x_skip = x
    x = tf.keras.layers.Conv2D(filter,(3,3), padding = 'Same')(x)
    x = tf.keras.layers.BatchNormalization(axis = 3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filter,(3,3), padding = 'Same')(x)
    x = tf.keras.layers.BatchNormalization(axis = 3)(x)
    x = tf.keras.layers.Add()([x,x_skip])
    return x

def convolutional_block(x,filter):
    x_skip = x
    x = tf.keras.layers.Conv2D(filter,(3,3), padding = 'Same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis = 3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filter,(3,3), padding = 'Same')(x)
    x = tf.keras.layers.BatchNormalization(axis = 3)(x)
    x_skip = tf.keras.layers.Conv2D(filter,(1,1),strides = (2,2))(x_skip)
    x = tf.keras.layers.Add()([x,x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x
    
def ResNet34 (input_shape):
    x_input = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.ZeroPadding2D((3,3))(x_input)
    x = tf.keras.layers.Conv2D(64,(7,7),strides = 2,padding = 'Same')(x)
    x = tf.keras.layers.BatchNormalization(axis = 3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(padding = 'Same',strides = 2, pool_size = 3)(x)
    block = (3,4,6,3)
    filter_size = 64
    for r in range(4):
        if r == 0:
            for j in range(block[r]-1):
                x = identity_block(x,filter_size)
        else:
            filter_size = 2*filter_size
            x = convolutional_block(x,filter_size)
            for j in range(block[r]-1):
                x = identity_block(x,filter_size)
    x = tf.keras.layers.AveragePooling2D((2,2),padding = 'Same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512,activation = 'relu')(x)
    x = tf.keras.layers.Dense(11,activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs= x_input, outputs = x , name = 'ResNet34')
    return model

In [5]:
input_shape = (36, 18, 1)
model = ResNet34(input_shape=input_shape)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.load_weights('cat11/.cp_model_4_pages/model')

## Train test split e fit

In [ ]:
digits = np.vstack((digits_1942_36,digits_1942_38,digits_1944_24,digits_1946_44))
labels = np.hstack((labels_1942_36,labels_1942_38,labels_1944_24,labels_1946_44))
print((digits.shape,labels.shape))

enc = OneHotEncoder(handle_unknown = 'ignore')

labels = pd.DataFrame(labels)
enc.fit(labels)
labels = enc.transform(labels).toarray()

X_train, X_test, y_train, y_test = train_test_split(digits, labels, test_size=0.2)

X_train = X_train/255.0
X_test = X_test/255.0

print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [24]:
model.fit(X_train,y_train,epochs = 10,validation_data = (X_test,y_test), verbose = 1)

Epoch 1/10
138/138 [==============================] - 122s 830ms/step - loss: 0.9867 - accuracy: 0.7309 - val_loss: 2.5274 - val_accuracy: 0.2158
Epoch 2/10
138/138 [==============================] - 114s 828ms/step - loss: 0.2412 - accuracy: 0.9262 - val_loss: 1.8758 - val_accuracy: 0.4572
Epoch 3/10
138/138 [==============================] - 114s 827ms/step - loss: 0.1742 - accuracy: 0.9490 - val_loss: 0.5567 - val_accuracy: 0.8352
Epoch 4/10
138/138 [==============================] - 117s 851ms/step - loss: 0.1173 - accuracy: 0.9640 - val_loss: 0.2319 - val_accuracy: 0.9362
Epoch 5/10
138/138 [==============================] - 115s 830ms/step - loss: 0.0803 - accuracy: 0.9786 - val_loss: 0.0994 - val_accuracy: 0.9690
Epoch 6/10
138/138 [==============================] - 115s 836ms/step - loss: 0.0293 - accuracy: 0.9939 - val_loss: 0.4000 - val_accuracy: 0.8953
Epoch 7/10
138/138 [==============================] - 115s 831ms/step - loss: 0.0866 - accuracy: 0.9754 - val_loss: 0.1299 -

## Visualizando resultados

In [27]:
def labeling(array):
            
    labels = pre_model.predict(array)
    labels = np.argmax(labels,axis = 1)
    
    algarismos_array = []
    posicao_algarismos = np.where(labels == 1)
    
    for k in range(len(labels)):
        if int(labels[k]) == 0:
            labels[k] = 11
    
    for i in range(len(posicao_algarismos[0])):
        algarismos_array.append(array[posicao_algarismos[0][i]])
        
    algarismos_array = np.array(algarismos_array)
    algarismos_array = algarismos_array.reshape(-1,36,18,1)

    labels_algarismos = model.predict(algarismos_array)
    labels_algarismos = np.argmax(labels_algarismos,axis = 1)
    
    for j in range(len(posicao_algarismos[0])):
        labels[posicao_algarismos[0][j]] = labels_algarismos[j]
    
    labels_string = []
    for i in range(len(labels)):
        if labels[i] in [0,1,2,3,4,5,6,7,8,9]:
            labels_string.append(str(labels[i]))
        if labels[i] == 11:
            labels_string.append(' ')
    
    #for k in range(len(labels_string)):
        #if int(labels_string[k]) == 11:
            #labels_string[k] = ' '
        
    return labels_string

def formating(array):
    formatedArray = []
    for i in range(len(array)):
        formatedArray.append(str(array[i]-1))
    
    for j in range(len(formatedArray)):
        if formatedArray[j] == '-1':
            formatedArray[j] = ' '
    return formatedArray   

def stats(predict,true):
    stats = []
    for i in range(len(true)):
        stats.append(true[i] == predict[i])
    return 'Qtd de acertos: {} / Porcentagem de acertos: {}'.format(stats.count(True),(stats.count(True)/len(true))*100)

In [51]:
predict_1945 = model.predict(digits_1945_38/255)
predict_1945 = np.argmax(predict_1945,axis = 1)

44/44 [==============================] - 2s 39ms/step


In [52]:
predict_1945 = formating(predict_1945)

In [53]:
stats(predict_1945,labels_1945_38)

'Qtd de acertos: 1350 / Porcentagem de acertos: 96.7741935483871'